# Data Setup

In [ ]:
# Import the necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

In [ ]:
# Set up the data generators for training and validation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split=0.2
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the image data
image_dir = './dtd-r1.0.1/dtd/images/'
class_names = os.listdir(image_dir)

X = []
y = []

for i, class_name in enumerate(class_names):
    class_dir = os.path.join(image_dir, class_name)
    for image_path in os.listdir(class_dir):
        image = keras.preprocessing.image.load_img(os.path.join(class_dir, image_path), target_size=(224, 224))
        image_array = keras.preprocessing.image.img_to_array(image)
        X.append(image_array)
        y.append(i)

# Data Pre-processing 

In [ ]:
# Convert the lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Count the total number of images 
total_images = len(X)
print("Total number of images:", total_images)

# Determine the image size and type
image_size = X[0].shape
image_type = type(X[1][0][0][0])
print("Image size:", image_size)
print("Image type:", image_type)

In [ ]:
# Check the shape of the data
print("X shape:", X.shape)
print("y shape:", y.shape)

# Check the number of classes
num_classes = len(class_names)
print("Number of classes:", num_classes)

# Check the distribution of classes
class_counts = np.bincount(y)
print("Class counts:", class_counts)

# Check the class distribution as percentages
class_percentages = class_counts / len(y) * 100
print("Class percentages:", class_percentages)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Reshape the input data to the desired shape
X_train = X_train.reshape(X_train.shape[0], 224, 224, 3)
X_test = X_test.reshape(X_test.shape[0], 224, 224, 3)

In [ ]:
# Perform one-hot encoding on the labels
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# LeNet 5 Model

In [ ]:
# Define the LeNet 5 model architecture
model = keras.Sequential([
    layers.Conv2D(6,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(16, kernel_size=(5,5), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])

# CNN Model

In [ ]:
# Define the CNN model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])

# VGG16 Model

In [ ]:
# Create a VGG16 model with pre-trained weights on ImageNet
vgg16 = keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

# Define the CNN model architecture with the VGG16 model as the base
model = keras.Sequential([
    vgg16,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    verbose=1,
    validation_data=(X_test, y_test)
)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])

# ResNet50 Model

In [ ]:
# Create a ResNet50 model with pre-trained weights on ImageNet
resnet50 = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling=None
)

# Define the CNN model architecture with ResNet50 backbone
model = keras.Sequential([
    resnet50,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    verbose=1,
    validation_data=(X_test, y_test)
)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])